In [1]:
# Params
input_size = (299,299)
batch_size = 100
num_workers = 4
learning_rate = 1e-03
momentum = 0.9
epochs = 3

# Training an image classifier

In [2]:
import torch

print("torch-version:", torch.__version__)
print("Available GPU:", torch.cuda.current_device())

torch-version: 1.1.0
Available GPU: 0


## 1. Loading and normalizing MH data

In [3]:
from dataset_MH import *

# 기존 데이터 경로
NORMAL_DIR = '../snapshots_pureok'
FAULT_DIR = '../snapshots'

# 추가 데이터 경로
ADD_NORMAL_DIR = '../snapshots_punch181'
ADD_FAULT_DIR = '../snapshots_punch181_NG'

# 카메라 번호 (1~4까지 있음, 카메라별로 각각 다 따로 학습해야됩니다.) CAM1 / CAM2 / CAM3 / CAM4
cam1 = 'CAM1'
cam2 = 'CAM2'
cam3 = 'CAM3'
cam4 = 'CAM4'

In [4]:
# 기존 데이터 로드
normal_list1 = filename_list(NORMAL_DIR, cam=cam1)
fault_list1 = filename_list(FAULT_DIR, cam=cam1)
normal_list2 = filename_list(NORMAL_DIR, cam=cam2)
fault_list2 = filename_list(FAULT_DIR, cam=cam2)
normal_list3 = filename_list(NORMAL_DIR, cam=cam3)
fault_list3 = filename_list(FAULT_DIR, cam=cam3)
normal_list4 = filename_list(NORMAL_DIR, cam=cam4)
fault_list4 = filename_list(FAULT_DIR, cam=cam4)

mhdb1 = MyungHwa_ansan_db(normal_list1, fault_list1, dataType='train', input_size=input_size, augmentation=True)
mhdb2 = MyungHwa_ansan_db(normal_list2, fault_list2, dataType='train', input_size=input_size, augmentation=True)
mhdb3 = MyungHwa_ansan_db(normal_list3, fault_list3, dataType='train', input_size=input_size, augmentation=True)
mhdb4 = MyungHwa_ansan_db(normal_list4, fault_list4, dataType='train', input_size=input_size, augmentation=True)

mhdb5 = MyungHwa_ansan_db(normal_list1, fault_list1, dataType='val', input_size=input_size, augmentation=True)
mhdb6 = MyungHwa_ansan_db(normal_list2, fault_list2, dataType='val', input_size=input_size, augmentation=True)
mhdb7 = MyungHwa_ansan_db(normal_list3, fault_list3, dataType='val', input_size=input_size, augmentation=True)
mhdb8 = MyungHwa_ansan_db(normal_list4, fault_list4, dataType='val', input_size=input_size, augmentation=True)
   
# 추가 데이터 로드
add_normal_list1 = filename_list(ADD_NORMAL_DIR, cam=cam1)
add_fault_list1 = filename_list(ADD_FAULT_DIR, cam=cam1)
add_normal_list2 = filename_list(ADD_NORMAL_DIR, cam=cam2)
add_fault_list2 = filename_list(ADD_FAULT_DIR, cam=cam2)
add_normal_list3 = filename_list(ADD_NORMAL_DIR, cam=cam3)
add_fault_list3 = filename_list(ADD_FAULT_DIR, cam=cam3)
add_normal_list4 = filename_list(ADD_NORMAL_DIR, cam=cam4)
add_fault_list4 = filename_list(ADD_FAULT_DIR, cam=cam4)

add_mhdb1 = MyungHwa_ansan_db(add_normal_list1, add_fault_list1, dataType='train', input_size=input_size, augmentation=True)
add_mhdb2 = MyungHwa_ansan_db(add_normal_list2, add_fault_list2, dataType='train', input_size=input_size, augmentation=True)
add_mhdb3 = MyungHwa_ansan_db(add_normal_list3, add_fault_list3, dataType='train', input_size=input_size, augmentation=True)
add_mhdb4 = MyungHwa_ansan_db(add_normal_list4, add_fault_list4, dataType='train', input_size=input_size, augmentation=True)

add_mhdb5 = MyungHwa_ansan_db(add_normal_list1, add_fault_list1, dataType='val', input_size=input_size, augmentation=True)
add_mhdb6 = MyungHwa_ansan_db(add_normal_list2, add_fault_list2, dataType='val', input_size=input_size, augmentation=True)
add_mhdb7 = MyungHwa_ansan_db(add_normal_list3, add_fault_list3, dataType='val', input_size=input_size, augmentation=True)
add_mhdb8 = MyungHwa_ansan_db(add_normal_list4, add_fault_list4, dataType='val', input_size=input_size, augmentation=True)
    
# 데이터셋 합치기
concat_mhdb1 = torch.utils.data.ConcatDataset([mhdb1, add_mhdb1])
concat_mhdb2 = torch.utils.data.ConcatDataset([mhdb2, add_mhdb2])
concat_mhdb3 = torch.utils.data.ConcatDataset([mhdb3, add_mhdb3])
concat_mhdb4 = torch.utils.data.ConcatDataset([mhdb4, add_mhdb4])

concat_mhdb5 = torch.utils.data.ConcatDataset([mhdb5, add_mhdb5])
concat_mhdb6 = torch.utils.data.ConcatDataset([mhdb6, add_mhdb6])
concat_mhdb7 = torch.utils.data.ConcatDataset([mhdb7, add_mhdb7])
concat_mhdb8 = torch.utils.data.ConcatDataset([mhdb8, add_mhdb8])


print(concat_mhdb1.__getitem__(0)[0].size(), concat_mhdb1.__len__())
print(concat_mhdb5.__getitem__(0)[0].size(), concat_mhdb5.__len__())

torch.Size([3, 299, 299]) 2545
torch.Size([3, 299, 299]) 424


In [5]:
mhdb_batch1 = torch.utils.data.DataLoader(concat_mhdb1, batch_size=batch_size, shuffle=True, num_workers=num_workers)
mhdb_batch2 = torch.utils.data.DataLoader(concat_mhdb2, batch_size=batch_size, shuffle=True, num_workers=num_workers)
mhdb_batch3 = torch.utils.data.DataLoader(concat_mhdb3, batch_size=batch_size, shuffle=True, num_workers=num_workers)
mhdb_batch4 = torch.utils.data.DataLoader(concat_mhdb4, batch_size=batch_size, shuffle=True, num_workers=num_workers)

val_mhdb_batch1 = torch.utils.data.DataLoader(concat_mhdb5, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_mhdb_batch2 = torch.utils.data.DataLoader(concat_mhdb6, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_mhdb_batch3 = torch.utils.data.DataLoader(concat_mhdb7, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_mhdb_batch4 = torch.utils.data.DataLoader(concat_mhdb8, batch_size=batch_size, shuffle=True, num_workers=num_workers)

## 2. Define a Inception net

### Important: In contrast to the other models the inception_v3 expects tensors with a size of N x 3 x 299 x 299, so ensure your images are sized accordingly.

In [6]:
import torchvision
print("torchvision-version:", torchvision.__version__)

net = torchvision.models.inception_v3(pretrained=False, aux_logits=True).cuda()

torchvision-version: 0.2.2


In [7]:
param_list = list(net.children())
print(param_list)

[BasicConv2d(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
), BasicConv2d(
  (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
), BasicConv2d(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
), BasicConv2d(
  (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
), BasicConv2d(
  (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
), InceptionA(
  (branch1x1): BasicConv2d(
    (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1),

## 3. Define a Loss function and optimizer

In [8]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

## 4. Train the network

In [9]:
times = 1

for epoch in range(epochs):  # loop over the dataset multiple times
    net.train()
    running_loss = 0.0
    for i, data in enumerate(mhdb_batch1):
        # get the inputs
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda().long()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs, aux_outputs = net(inputs)
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        
        loss = loss1 + 0.4*loss2
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    if epoch % times == 0:
        print('[%d] loss: %.3f' %(epoch, running_loss / times))
        running_loss = 0.0
        #torch.save(model,'./data/cifar_model.pkl')

print('\nFinished Training')

[0] loss: 172.047
[1] loss: 50.102
[2] loss: 37.048

Finished Training


## 5. Test the network on the test data

In [11]:
correct = 0
total = 0
cpu_net = net.cpu()

with torch.no_grad():
    cpu_net.eval()
    for data in val_mhdb_batch1:
        images, labels = data
        images = images.cpu()
        labels = labels.cpu().long()
        
        outputs = cpu_net(images)
        _, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' %(100 * correct / total)) #more than 50 percent

Accuracy of the network on the 10000 test images: 65 %
